# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [109]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [110]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [26]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [55]:
# Housing Units 
def housing_units_per_year():
    df_housing_units = sfo_data.groupby('year')
    df_housing_units = df_housing_units['housing_units'].mean()
    df_housing_units = pd.DataFrame(df_housing_units)
    maximum_housing = max(df_housing_units['housing_units']) + 1000
    minimum_housing = min(df_housing_units['housing_units']) - 1000
    
    avg_graph = df_housing_units.hvplot.bar(
        label = "Average Housing Units by Year 2010-2016",
        ylim = (minimum_housing, maximum_housing),
        width = 450
    )
    
    return avg_graph

In [63]:
# Average Gross Rent 
def average_gross_rent():
    df_gross_rent = sfo_data.groupby('year')
    df_gross_rent = df_gross_rent['gross_rent'].mean()
   
    avg_rent = df_gross_rent.hvplot(
        label="Average Gross Rent in San Francisco", 
        ylabel= 'Average Rent', 
        xlabel = 'Year',
    ).opts(width=450)
    
    return avg_rent
    

In [115]:
# Average Sales Price 
def average_sales_price():
    sale_price_sqft = sfo_data.groupby('year')
    sale_price_sqft = sale_price_sqft['sale_price_sqr_foot'].mean()
    
    avg_price = sale_price_sqft.hvplot(
        label= "Average Sale Price Per Square Foot in San Francisco", 
        ylabel='Avg. Sale Price', 
        xlabel= 'Year').opts(width=450)
    
    return avg_price

In [77]:
# Average Price By Neighborhood 

def average_price_by_neighborhood():
    neighborhood_price = sfo_data.groupby(['year', 'neighborhood']).mean()

    avg_neighborhood = neighborhood_price.hvplot.line(
        "year",
        "sale_price_sqr_foot",
        xlabel="Year", 
        ylabel="Sale Price Per SQFT", 
        groupby="neighborhood",
        ).opts(width=450)

    return avg_neighborhood

In [172]:
# Top 10 Most Expensive Neighborhoods 
def top_most_expensive_neighborhoods():
    mean_sale = sfo_data.groupby('neighborhood').mean()
    mean_sale = mean_sale.sort_values(by='sale_price_sqr_foot', ascending = False)
    expensive_sale = mean_sale.head(10)

    top_neighborhoods_sqft = expensive_sale['sale_price_sqr_foot']
    top_sqft = top_neighborhoods_sqft.hvplot.bar(rot=60,
                            label = 'Top 10 Expensive Neighborhoods by SQFT',
                             ylabel = 'Price per SQFT',
                             x='neighborhood',
                             xlabel = 'Neighborhood',
                             ylim = (550, 950),
                            ).opts(width=450, hover_line_color = 'red')
    return top_sqft

In [173]:
# Parallel Coordinates
def parallel_coordinates():
    mean_sale = sfo_data.groupby('neighborhood').mean()
    mean_sale = mean_sale.sort_values(by='sale_price_sqr_foot', ascending = False)
    expensive_sale = mean_sale.head(10)
    
    parallel_coor = px.parallel_coordinates(expensive_sale, color = 'sale_price_sqr_foot', title = 'Top Neighborhoods')
    
    return parallel_coor


In [174]:
mean_sale = sfo_data.groupby('neighborhood').mean()
mean_sale = mean_sale.sort_values(by='sale_price_sqr_foot', ascending = False)
expensive_sale = mean_sale.head(10)
expensive_sale = expensive_sale.reset_index()
expensive_sale

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,Union Square District,903.993258,377427.50,2555.166667
1,Merced Heights,788.844818,380348.00,3414.000000
2,Miraloma Park,779.810842,375967.25,2155.250000
3,Pacific Heights,689.555817,378401.00,2817.285714
4,Westwood Park,687.087575,382295.00,3959.000000
5,Telegraph Hill,676.506578,378401.00,2817.285714
6,Presidio Heights,675.350212,378401.00,2817.285714
7,Cow Hollow,665.964042,378401.00,2817.285714
8,Potrero Hill,662.013613,378401.00,2817.285714
9,South Beach,650.124479,375805.00,2099.000000


In [175]:
# Parallel Categories 
def parallel_categories():
    mean_sale = sfo_data.groupby('neighborhood').mean()
    mean_sale = mean_sale.sort_values(by='sale_price_sqr_foot', ascending = False)
    mean_sale = mean_sale.reset_index()
    expensive_sale = mean_sale.head(10)
    
    parallel_cat = px.parallel_categories(expensive_sale,
                        dimensions = ['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
                        color = 'sale_price_sqr_foot', 
                       labels={
                           "neighborhood": 'Neighborhood',
                           'sale_price_sqr_foot': 'Price Per SQFT',
                           'housing_units': 'Housing Units',
                           'gross_rent': 'Gross Rent',
                       },
                       )
    return parallel_cat

In [176]:
# Neighborhood Map 

def neighborhood_map():
    mean_sale = sfo_data.groupby('neighborhood').mean()
    mean_sale = mean_sale.sort_values(by='sale_price_sqr_foot', ascending = False)
    mean_sale = mean_sale.reset_index()
    mean_sale.rename(columns = {'neighborhood':'Neighborhood'}, inplace=True)
    neighborhoods_ct = pd.merge(mean_sale, df_neighborhood_locations, left_on = 'Neighborhood', right_on = 'Neighborhood')


    mapbox_plot = px.scatter_mapbox(
        neighborhoods_ct,
        lat='Lat',
        lon='Lon',
        size='sale_price_sqr_foot',
        color='gross_rent',
        title="Average Price per SQFT and Gross Rent",
        zoom = 10
        )
    return mapbox_plot

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [177]:
row_of_rent_sqft = pn.Row(average_gross_rent, average_sales_price)

In [178]:
row_of_housing_units = pn.Row(housing_units_per_year)

In [179]:
columns_expensive = pn.Column(average_price_by_neighborhood, top_most_expensive_neighborhoods)

In [183]:
first_tab = pn.Column(parallel_categories, parallel_coordinates) 

In [187]:
mapbox_map = pn.Row(neighborhood_map)

In [192]:
panel = pn.Tabs(
    ("Welcome", mapbox_map),
    ("Top Neighborhoods", first_tab),
    ("Average Prices", row_of_rent_sqft),
    ("Housing Units", row_of_housing_units),
    ('Neighborhood Prices', columns_expensive),
    ) 
panel
    

Tabs
    [0] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive24342')
    [1] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive23376')
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive23382')
    [2] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive21726')
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive21861')
    [3] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive21997')
    [4] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive22089')
                    [0] HoloViews(DynamicMap, name='interactive22089')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive22137')

## Serve the Panel Dashboard

In [191]:
panel.servable()

Tabs
    [0] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive24342')
    [1] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive23376')
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive23382')
    [2] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive21726')
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive21861')
    [3] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive21997')
    [4] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive22089')
                    [0] HoloViews(DynamicMap, name='interactive22089')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive22137')